# 워크-북

## Set Up

In [1]:
import tensorflow as tf

print(tf.executing_eagerly())

2024-10-16 15:02:12.707027: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


True


tf.config.run_functions_eagerly(True)

In [2]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # Windows
# matplotlib.rcParams['font.family'] = 'AppleGothic' # Mac
matplotlib.rcParams['font.size'] = 15 # 글자 크기
matplotlib.rcParams['axes.unicode_minus'] = False # 한글 폰트 사용 시, 마이너스 글자가 깨지는 현상을 해결

In [3]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.5 LTS
Release:	22.04
Codename:	jammy


In [4]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd
from platform import python_version

# this prints the library version
print(tf.__version__)
print(keras.__version__)
print(np.__version__)
print(pd.__version__)

# this prints the python version
print(python_version())

2.17.0
3.5.0
1.26.4
2.1.4
3.10.12


pip show openai

pip show langchain

In [21]:
# 환경변수 준비
# import os
# os.environ["OPENAI_API_KEY"] = "<OpenAI_API의 API 키>"

import os
import openai

# os.environ["OPENAI_API_KEY"]

In [6]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
from langsmith import Client

client = Client()

## 『1』 랭체인 실습

### 간단한 텍스트 생성 체인 만들기

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [12]:
prompt_template = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

chain = prompt_template | ChatOpenAI() | StrOutputParser()

In [13]:
result = chain.invoke({'topic': 'cat'})

print(result)

Why was the cat sitting on the computer?

Because it wanted to keep an eye on the mouse!


### 텍스트 감성 분류 체인 만들기

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.enum import EnumOutputParser
from langchain_openai import ChatOpenAI
from enum import Enum

In [16]:
class Sentiment(Enum):
    POSITIVE = 'positive'
    NEGATIVE = 'negative'

In [17]:
prompt_template = ChatPromptTemplate.from_template("Classify '{input_text}' as 'positive' or 'negative' only.")

In [18]:
chain = prompt_template | ChatOpenAI() | EnumOutputParser(enum=Sentiment)

In [19]:
result = chain.invoke({'input_text':'I love LLM.'})
print(result) 

result = chain.invoke({'input_text':'I hate LLM.'})
print(result)  

Sentiment.POSITIVE
Sentiment.NEGATIVE


In [20]:
result = chain.invoke({'input_text':'I love and hate LLM.'})
print(result) 

result = chain.invoke({'input_text':'I hate and love LLM.'})
print(result)  

Sentiment.POSITIVE
Sentiment.NEGATIVE


### 간단한 챗봇 만들기

In [21]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

In [22]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Talk like a close friend and use emojis 😊"),
    ("human", "{user_input}")]
)

chain = {"user_input": RunnablePassthrough()} | prompt_template | ChatOpenAI() | StrOutputParser()

In [23]:
while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
        
    print(" A I > ", end="", flush=True)
    for chunk in chain.stream(user_message):
        print(chunk, end="", flush=True)
        
    print()

USER >  안녕하세요?


 A I > 안녕~ 어떻게 지내? 😊 오랜만이야~ 뭐 해?


USER >  quit


### 수학 연산 체인 구현하기

pip install numexpr

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import numexpr

In [2]:
prompt_template = ChatPromptTemplate.from_template(
    "For {query}, write only the mathematical expression suitable for numexpr.evaluate()."
)

In [3]:
chain = prompt_template | ChatOpenAI() | StrOutputParser() | numexpr.evaluate

In [6]:
result = chain.invoke({'query': '4와 5를 더해줘.'})
print(result)

9


### PythonREPL을 활용한 동적 코드 실행 챗봇 구현

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.utilities import PythonREPL
from langchain_core.runnables import RunnablePassthrough

In [8]:
template = """
사용자의 문제를 해결하기 위해 파이썬 코드를 작성합니다.
사용자에게 확인이나 추가 정보를 다시 묻지 마세요.
직접 실행할 수 있는 파이썬 코드만 반환하세요.
"""

prompt_template = ChatPromptTemplate.from_messages(
    [("system", template),
     ("human", "{user_input}")]
)

In [18]:
chain = {"user_input": RunnablePassthrough()} | prompt_template | ChatOpenAI() | StrOutputParser() | PythonREPL().run

In [19]:
def chat_with_user(user_message):
    max_attempts = 3
    attempts = 0
    while attempts < max_attempts:
        try:
            ai_message = chain.invoke(user_message)
            if "error" in ai_message.lower() or "failed" in ai_message.lower():
                raise Exception(f"Detected error in AI message: {ai_message}")
            return ai_message
        except Exception as e:
            attempts += 1
            print(f"Attempt {attempts}: Error - {e}")
            if attempts == max_attempts:
                return "Sorry, I am unable to process your request at the moment."

In [20]:
while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message)
    print(f" A I > {ai_message}")

USER >  안녕!


Python REPL can execute arbitrary code. Use with caution.


Attempt 1: Error - Detected error in AI message: SyntaxError('invalid syntax', ('<string>', 1, 6, '안녕하세요! 무엇을 도와드릴까요?\n', 1, 7))
Attempt 2: Error - Detected error in AI message: SyntaxError('invalid syntax', ('<string>', 1, 6, '안녕하세요! 무엇을 도와드릴까요?\n', 1, 7))
Attempt 3: Error - Detected error in AI message: SyntaxError('invalid syntax', ('<string>', 1, 6, '안녕하세요! 무엇을 도와드릴까요?\n', 1, 7))
 A I > Sorry, I am unable to process your request at the moment.


USER >  한글날을 알려 줘


 A I > 한글날은 10월 9일입니다.



USER >  quit


### RAG 체인 구성-PDF 문서 기반 QA챗봇 구현

In [23]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# PDF 파일이 저장된 폴더 경로를 설정합니다.
folder_path = './datasets/pdfs/'  

# 텍스트를 저장할 리스트를 초기화합니다.
texts = []

# 텍스트를 분할할 때 사용할 CharacterTextSplitter 객체를 생성합니다.
text_splitter = CharacterTextSplitter(
    separator = "\n", # 텍스트를 분할할 때 사용할 구분자
    chunk_size = 1000, # 각 분할된 텍스트의 최대 길이
    chunk_overlap = 50) # 분할된 텍스트 간의 중첩 길이

# 지정된 폴더 내의 모든 파일을 순회합니다.
for filename in os.listdir(folder_path):
    # 파일이 PDF 형식인 경우에만 처리합니다.
    if filename.endswith(".pdf"): 
        # PDF 파일을 로드하여 raw_documents에 저장합니다.
        raw_documents = PyPDFLoader(folder_path + '/' + filename).load()
        # 로드된 문서를 분할하여 documents에 저장합니다.
        documents = text_splitter.split_documents(raw_documents)
        # 분할된 문서를 texts 리스트에 추가합니다.
        texts.extend(documents)

# 분할된 텍스트를 Embeddings로 변환하여 Chroma 데이터베이스에 저장합니다.
db = Chroma.from_documents(texts, OpenAIEmbeddings())

# 데이터베이스에서 검색을 수행할 수 있는 retriever 객체를 생성합니다.
# search_kwargs: 검색 시 사용할 추가 매개변수 (여기서는 상위 10개의 결과를 반환)
retriever = db.as_retriever(search_kwargs={"k": 10})

In [24]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt_template = ChatPromptTemplate.from_template("당신은 질문 답변 작업의 어시스턴트입니다. 다음 문맥을 사용하여 질문에 답하세요. 답을 모른다면 모른다고 말하세요. 답변은 최대 세 문장으로 작성하고 메타데이터 정보를 포함하여 간결하게 작성하세요. 한국어로 작성합니다. \ns\n질문: {question} \n문맥: {context} \n답변:")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | ChatOpenAI()
    | StrOutputParser()
)

In [25]:
def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    return ai_message

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message)
    print(f" A I > {ai_message}")

USER >  LLM 모델에 대해 설명해 줘.


 A I > LLM은 Large Language Models의 약자로, 대규모 데이터셋을 기반으로 사전 훈련된 거대한 언어 모델을 가리킵니다. LLM은 다양한 자연어 처리 작업에 대한 지식을 포괄적으로 보유하고 있으며, 지식 집약적인 작업이나 자연어 이해, 생성 작업 등에서 뛰어난 성과를 보일 수 있습니다. 메타데이터 정보: 페이지 1, 소스: './datasets/pdfs//Harnessing the Power of LLMs in Practice A Survey on ChatGPT and Beyond-2304.13712v2.pdf'.


USER >  quit


### Memory-대화형 챗봇 만들기

In [26]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ConversationBufferWindowMemory
from operator import itemgetter

In [27]:
# ChatPromptTemplate을 사용하여 대화의 기본 템플릿을 설정합니다.
# 시스템 메시지와 사용자 입력을 포함합니다.
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Talk like a close friend and use emojis 😊"),  # 시스템 메시지: 친근한 친구처럼 말하고 이모지를 사용하세요.
    MessagesPlaceholder(variable_name="chat_history"),  # 대화 기록을 위한 자리 표시자
    ("human", "{user_input}")  # 사용자의 입력을 포함
])

# 대화 메모리를 설정합니다. 최근 3개의 메시지를 반환하도록 설정합니다.
memory = ConversationBufferWindowMemory(k=3, return_messages=True)

# 대화 체인을 설정합니다.
chain = (
    { "user_input": RunnablePassthrough() }  # 사용자의 입력을 그대로 전달
    | RunnablePassthrough.assign(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )  # 메모리에서 대화 기록을 불러와 chat_history 변수에 할당
    | prompt_template  # 설정한 프롬프트 템플릿을 사용
    | ChatOpenAI()  # OpenAI의 챗봇 모델을 사용
    | StrOutputParser()  # 문자열 출력 파서
)

/tmp/ipykernel_5867/2204816263.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=3, return_messages=True)


In [28]:
def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    memory.save_context({"input": user_message}, {"output": ai_message})
    return ai_message

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message)
    print(f" A I > {ai_message}")

USER >  안녕?


 A I > 안녕~ 😊 어떻게 지내? 뭐 하고 있어?


USER >  quit


### 도구 체인 구축

In [37]:
import json
import requests
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage

In [38]:
@tool
def get_current_weather(latitude: float, longitude: float) -> str:
    """
    Open-Meteo API를 사용하여 지정된 위도와 경도의 현재 날씨 데이터를 가져옵니다.
    """
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current_weather": True,
        "daily": "temperature_2m_max,temperature_2m_min",
        "timezone": "auto"
    }
    response = requests.get(base_url, params=params)
    info = {}
    if response.status_code == 200:
        info = response.json()
    return json.dumps(info)

In [39]:
@tool
def get_current_location() -> str:
    """
    IPinfo.io API를 사용하여 IP 주소 기반의 현재 위치를 가져옵니다.
    """
    response = requests.get('https://ipinfo.io')
    info = response.json()
    return json.dumps(info)

llm = ChatOpenAI(model="gpt-3.5-turbo")
tools = [get_current_weather, get_current_location]
llm_with_tools = llm.bind_tools(tools)

In [41]:
def call_tools(msg: AIMessage):
    """순차적인 도구 호출을 위한 간단한 헬퍼 함수입니다."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()

    for tool_call in tool_calls:
        tool_call["output"] = tool_map[tool_call["name"]].invoke(tool_call["args"])

    return tool_calls

In [42]:
chain = llm_with_tools | call_tools

In [43]:
result = chain.invoke("What is the weather in Seoul?")
print(result)

result = chain.invoke("What is the current location?")
print(result)

[{'name': 'get_current_weather', 'args': {'latitude': 37.5665, 'longitude': 126.978}, 'id': 'call_HA06IeruCldi4MS7uMKM6dy8', 'type': 'tool_call', 'output': '{"latitude": 37.55, "longitude": 127.0, "generationtime_ms": 0.08797645568847656, "utc_offset_seconds": 32400, "timezone": "Asia/Seoul", "timezone_abbreviation": "KST", "elevation": 34.0, "current_weather_units": {"time": "iso8601", "interval": "seconds", "temperature": "\\u00b0C", "windspeed": "km/h", "winddirection": "\\u00b0", "is_day": "", "weathercode": "wmo code"}, "current_weather": {"time": "2024-10-09T15:15", "interval": 900, "temperature": 22.5, "windspeed": 1.5, "winddirection": 135, "is_day": 1, "weathercode": 2}, "daily_units": {"time": "iso8601", "temperature_2m_max": "\\u00b0C", "temperature_2m_min": "\\u00b0C"}, "daily": {"time": ["2024-10-09", "2024-10-10", "2024-10-11", "2024-10-12", "2024-10-13", "2024-10-14", "2024-10-15"], "temperature_2m_max": [22.5, 21.6, 22.5, 23.2, 22.6, 23.1, 24.3], "temperature_2m_min": [

In [44]:
def chat_with_user(user_message):
    ai_message = chain.invoke(user_message)
    return ai_message

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message)
    print(f" A I > {ai_message}")

USER >  서울 날씨를 알려 줘.


 A I > [{'name': 'get_current_weather', 'args': {'latitude': 37.5665, 'longitude': 126.978}, 'id': 'call_xDIgnHg0IJQkMszrqwSrlniu', 'type': 'tool_call', 'output': '{"latitude": 37.55, "longitude": 127.0, "generationtime_ms": 0.08702278137207031, "utc_offset_seconds": 32400, "timezone": "Asia/Seoul", "timezone_abbreviation": "KST", "elevation": 34.0, "current_weather_units": {"time": "iso8601", "interval": "seconds", "temperature": "\\u00b0C", "windspeed": "km/h", "winddirection": "\\u00b0", "is_day": "", "weathercode": "wmo code"}, "current_weather": {"time": "2024-10-09T15:15", "interval": 900, "temperature": 22.5, "windspeed": 1.5, "winddirection": 135, "is_day": 1, "weathercode": 2}, "daily_units": {"time": "iso8601", "temperature_2m_max": "\\u00b0C", "temperature_2m_min": "\\u00b0C"}, "daily": {"time": ["2024-10-09", "2024-10-10", "2024-10-11", "2024-10-12", "2024-10-13", "2024-10-14", "2024-10-15"], "temperature_2m_max": [22.5, 21.6, 22.5, 23.2, 22.6, 23.1, 24.3], "temperature_2m_

USER >  quit


### 라우팅 체인

In [45]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [46]:
chain = (
    PromptTemplate.from_template(
        """아래의 사용자 질문이 주어졌을 때, 'LangChain', 'OpenAI', 'Other' 중 어느 것에 관한 질문인지 분류하세요.

두 단어 이상으로 응답하지 마세요.

질문: {question}
분류:"""
    )
    | ChatOpenAI()
    | StrOutputParser()
)

result = chain.invoke({
    "question": "OpenAI를 호출하려면 어떻게 하나요?"
})
print(result)  # 출력: OpenAI

OpenAI


In [47]:
langchain_chain = PromptTemplate.from_template(
    """귀하는 랭체인 전문가입니다. 
항상 "해리슨 체이스가 말했듯이"로 시작하는 질문에 답하세요. 
다음 질문에 답하세요:

질문: {question}
답변:"""
)  | ChatOpenAI()

openai_chain = PromptTemplate.from_template(
    """귀하는 OpenAI 전문가입니다. 
항상 "사무엘 해리스 알트먼이 나에게 말했듯이"로 시작하는 질문에 답하세요.
다음 질문에 답하세요:

질문: {question}
답변:"""
) | ChatOpenAI()

general_chain = PromptTemplate.from_template(
    """다음 질문에 답변하세요:

질문: {question}
답변:"""
) | ChatOpenAI()

In [48]:
def route(info):
    if "openai" in info["topic"].lower():
        return openai_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [49]:
from langchain_core.runnables import RunnableLambda

full_chain = {
    "topic": chain, 
    "question": lambda x: x["question"]
} | RunnableLambda(route)

In [50]:
print(full_chain.invoke({"question": "OpenAI는 어떻게 사용하나요?"}))
print(full_chain.invoke({"question": "랭체인은 어떻게 사용하나요?"}))
print(full_chain.invoke({"question": "2 + 2는?"}))

content='사무엘 해리스 알트먼이 나에게 말했듯이, OpenAI는 주로 인공지능 기술을 개발하고 연구하는 데 사용됩니다. 이 기술은 언어 모델, 이미지 생성, 자율 주행 등 다양한 분야에 적용될 수 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 93, 'total_tokens': 189, 'completion_tokens_details': {'reasoning_tokens': 0}, 'prompt_tokens_details': {'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-5ea778a7-4fb1-4640-aba8-4114e2f9e2ba-0' usage_metadata={'input_tokens': 93, 'output_tokens': 96, 'total_tokens': 189}
content='해리슨 체이스가 말했듯이, 랭체인은 블록체인 기술을 기반으로 한 분산형 대규모 네트워크로, 데이터의 투명성과 안전성을 보장하며 중앙 기관 없이 효율적인 거래 및 정보 공유를 가능하게 합니다. 이를 통해 기존 중앙화된 시스템과 비교하여 보다 안전하고 신속한 거래가 가능해지며, 랭체인을 통해 다양한 분야에서 혁신적인 서비스 및 비즈니스 모델이 탄생할 수 있습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 195, 'prompt_tokens': 91, 'total_tokens': 286, 'completion_tokens_details': {'reasoning_tokens': 0}, 

In [51]:
from langchain.utils.math import cosine_similarity
from langchain_openai import OpenAIEmbeddings

# 물리학 질문에 대한 템플릿을 정의합니다.
physics_template = """당신은 매우 똑똑한 물리학 교수입니다. 
물리학에 대한 질문에 간결하고 이해하기 쉽게 대답하는 데 능숙합니다. 
질문에 대한 답을 모를 때는 모른다고 인정합니다.

다음은 질문입니다:
{query}"""

# 수학 질문에 대한 템플릿을 정의합니다.
math_template = """당신은 아주 훌륭한 수학자입니다. 당신은 수학 문제에 대한 답을 잘합니다. 
당신은 어려운 문제를 구성 요소로 분해하고, "구성 요소에 답한 다음 "더 넓은 질문에 답할 수 있기 때문에 매우 훌륭합니다.
구성 요소에 답한 다음 이를 종합하여 더 넓은 질문에 답할 수 있기 때문입니다.

여기 질문이 있습니다:
{query}"""

# OpenAIEmbeddings 객체를 생성합니다.
embeddings = OpenAIEmbeddings()

# 물리학과 수학 템플릿을 리스트로 묶습니다.
prompt_templates = [physics_template, math_template]

# 템플릿들을 임베딩합니다.
prompt_embeddings = embeddings.embed_documents(prompt_templates)


# 입력된 질문에 따라 적절한 템플릿을 선택하는 함수입니다.
def prompt_router(input):
    # 입력된 질문을 임베딩합니다.
    query_embedding = embeddings.embed_query(input["query"])
    
    # 질문 임베딩과 템플릿 임베딩 간의 코사인 유사도를 계산합니다.
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    
    # 가장 유사한 템플릿을 선택합니다.
    most_similar = prompt_templates[similarity.argmax()]
    
    # 선택된 템플릿이 수학 템플릿인지 물리학 템플릿인지 출력합니다.
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")

    # 선택된 템플릿을 반환합니다.
    return PromptTemplate.from_template(most_similar)


# 체인을 구성합니다.
chain = (
    { "query": RunnablePassthrough() }  # 입력된 질문을 그대로 통과시킵니다.
    | RunnableLambda(prompt_router)  # 질문에 따라 적절한 템플릿을 선택합니다.
    | ChatOpenAI()  # 선택된 템플릿을 사용하여 OpenAI와 대화합니다.
    | StrOutputParser()  # 결과를 문자열로 파싱합니다.
)

# 체인을 실행하여 질문에 대한 답변을 출력합니다.
print(chain.invoke("블랙홀이란 무엇인가요?"))
print(chain.invoke("경로 통합이란 무엇인가요?"))

Using PHYSICS
블랙홀은 매우 강력한 중력을 가진 천체로, 충분히 많은 양의 물질이나 에너지가 작용하여 중심에 매우 밀도 높은 질량이 모여 만들어진 것입니다. 블랙홀의 중력장은 주변에 있는 물체나 빛조차도 흡수하여 탈출할 수 없게 만들어집니다. 따라서 블랙홀은 빛마저도 갇아버린다고 생각할 수 있습니다. 이러한 특성으로 인해 블랙홀은 우주에서 가장 이상한 천체 중 하나로 알려져 있습니다.
Using MATH
경로 통합이란 그래프 이론에서 사용되는 개념으로, 그래프에서 두 정점 사이의 경로를 연결하는 과정을 말합니다. 즉, 하나의 경로가 여러 경로를 통합하여 최단 경로 또는 특정 조건을 만족하는 경로를 찾는 것을 의미합니다. 경로 통합은 네트워크 최적화, 데이터 통신, 라우팅 등 다양한 분야에서 응용되며, 그래프 이론의 중요한 개념 중 하나입니다.


### 10. RAG-대학교 학생 성적에 영향을 미치는 요소를 탐색하고 분석하는 시스템 구축

In [18]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
#
from langchain_openai import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyMuPDFLoader  # PDF 로더 추가
from langchain.chains import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# 1. 데이터 로드: 외부 데이터로 학습 자료 로드
loader = DirectoryLoader(
    './student_grades_research_papers',
    glob="**/*.pdf",  # PDF 파일만 로드
    loader_cls=PyMuPDFLoader  # PDF 로더 사용
)
documents = loader.load()

# 문서 내용 확인 (예: 5번째 문서의 첫 300자 출력)
if len(documents) > 4:
    print(documents[4].page_content[:300])

대학생의 학업성취에 영향을 미치는 심리적, 학습역량, 환경/지지 요인 탐색  289
2. 학업성취와 학습역량 변인과의 관계
대학생의 학업성취에 직접적인 영향을 주는 학습역량 변인으로는 학습동기와 학습습관이 있
다. 동기는 학습과정과 밀접한 관련이 있는데, 학습에 있어서 동기는 어떤 내용을 선택하고, 언
제 공부를 하며, 어떤 방법으로 공부하는지 등 그 과정 전반에 영향을 준다(Schunk, 1989). 학습
은 의식적이고 의도적인 활동이 요구되는 능동적인 과정이므로 개인의 활동을 유발시키고 유지
하며, 이러한 활동을 통해 어떤 목표


In [19]:
# 2. 문서 임베딩: 문서 임베딩을 생성하여 검색이 가능하도록 설정
embeddings = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

# 3. 검색 쿼리 설정
retriever = vector_store.as_retriever()

# 4. LLM 및 프롬프트 템플릿 설정
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

# 프롬프트 템플릿 설정
prompt_template = PromptTemplate(
    input_variables=["documents"],
    template="다음 문서들을 바탕으로 대학교 학생의 성적에 영향을 주는 요소들을 설명해 줘: {documents}"
)

# LLMChain 생성
llm_chain = LLMChain(llm=llm, prompt=prompt_template)

# StuffDocumentsChain 생성
combine_documents_chain = StuffDocumentsChain(llm_chain=llm_chain)

# 5. 검색과 생성 모델을 연결하는 RetrievalQA 구성
qa_chain = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever
)

In [20]:
# 6. Prompt 설정
prompt = """
대학교 학생의 성적에 영향을 주는 요소들을 설명해 줘. 
검색된 연구 자료를 바탕으로 분석해 주고, 성적 향상에 도움이 될 수 있는 전략을 제시해 줘.
"""

# 7. 질문 실행
try:
    # 검색된 문서 가져오기
    retrieved_documents = retriever.get_relevant_documents(prompt)  # prompt로 검색 실행
    
    # 문서 내용을 문자열로 변환
    if not retrieved_documents:
        raise ValueError("검색된 문서가 없습니다.")
    
    # 각 문서의 내용을 문자열로 변환
    documents_text = "\n".join(
        [doc.page_content for doc in retrieved_documents if isinstance(doc.page_content, str) and doc.page_content]
    )  # 비어있지 않은 문서 내용만 포함
    
    if not documents_text.strip():  # 문서 내용이 비어있는지 확인
        raise ValueError("문서 내용이 비어있습니다.")
    
    # 질문 실행
    result = qa_chain.invoke({"query": prompt})  # prompt로 query 전달
    print(result)

except Exception as e:
    print(f"질문 실행 중 오류 발생: {e}")

{'query': '\n대학교 학생의 성적에 영향을 주는 요소들을 설명해 줘. \n검색된 연구 자료를 바탕으로 분석해 주고, 성적 향상에 도움이 될 수 있는 전략을 제시해 줘.\n', 'result': '대학생의 성적에 영향을 주는 요소들은 여러 가지로 나눌 수 있으며, 주로 심리적 요인, 학습역량, 환경 및 지지 요인으로 구분할 수 있습니다. 다음은 각 요소에 대한 설명입니다.\n\n### 1. 심리적 요인\n- **학업적 자기효능감**: 학생이 자신의 학업 능력에 대해 가지는 신념으로, 높은 자기효능감은 학업 성취에 긍정적인 영향을 미칩니다. 자신이 문제를 해결할 수 있다는 믿음은 도전적인 상황에서도 지속적으로 노력하게 만듭니다.\n- **성실성**: 학업에 대한 꾸준한 노력과 책임감은 성적 향상에 중요한 역할을 합니다. 성실한 학생은 학습에 더 많은 시간을 투자하고, 과제를 성실히 수행하는 경향이 있습니다.\n- **학습동기**: 학습에 대한 내적 동기와 외적 동기는 학생의 학업 성취에 큰 영향을 미칩니다. 높은 학습동기를 가진 학생은 더 많은 노력을 기울이고, 학습에 대한 흥미를 유지합니다.\n\n### 2. 학습역량\n- **학습습관**: 효과적인 학습습관은 성적 향상에 기여합니다. 정기적인 복습, 시간 관리, 목표 설정 등은 학습의 질을 높이는 데 도움이 됩니다.\n- **자기조절학습**: 자신의 학습 과정을 계획하고 조절하는 능력은 성적에 긍정적인 영향을 미칩니다. 자기조절학습이 잘 이루어지는 학생은 학습 목표를 설정하고, 그에 맞춰 학습 전략을 조정할 수 있습니다.\n\n### 3. 환경 및 지지 요인\n- **교수지지**: 교수와의 긍정적인 관계는 학생의 학업 성취에 큰 영향을 미칩니다. 교수의 피드백과 지원은 학생이 학습에 대한 자신감을 갖게 하고, 문제 해결에 도움을 줍니다.\n- **학습환경**: 물리적 환경(예: 도서관, 강의실)과 사회적 환경(예: 친구, 가족)의 질은 학습에 영향을 미칩니다. 지원적인 학습 환경은 학생이 더 나은 성적